# Use BERT Representations with LogisticRegression Softmax Classifier

In [1]:
from collections import Counter
import os
import numpy as np
import pandas as pd
import torch
from torch import nn, optim
import torch.nn as nn
from torch.utils.data import TensorDataset, Dataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from transformers import BertTokenizer, BertModel, BertForSequenceClassification


import dataset
import vsm
import sst

In [2]:
TWITTER = 2
TWITTER_AIRLINES = 3
TWITTER_APPLE = 4

In [3]:
twitter_train, twitter_validate, twitter_test =  dataset.dataset_reader(TWITTER)
[twitter_train, twitter_validate, twitter_test] = list(map(lambda ds : dataset.prune_columns(2, ds), [twitter_train, twitter_validate, twitter_test]))

In [4]:
airline_train, airline_validate, airline_test =  dataset.dataset_reader(TWITTER_AIRLINES)
[airline_train, airline_validate, airline_test] = list(map(lambda ds : dataset.prune_columns(3, ds), [airline_train, airline_validate, airline_test]))

In [5]:
apple_train, apple_validate, apple_test =  dataset.dataset_reader(TWITTER_APPLE)
[apple_train, apple_validate, apple_test] = list(map(lambda ds : dataset.prune_columns(4, ds), [apple_train, apple_validate, apple_test]))

In [6]:
# Rename labels for consistency
def rename_sentiment(sentiment):
        if sentiment in ["5", "positive"]:
            return "Positive"
        elif sentiment in ["3", "neutral"]:
            return "Neutral"
        elif sentiment in ["1", "negative"]:
            return "Negative"
        elif sentiment in ["not_relevant","Irrelevant"]:
            return "Neutral"
        else:
            return sentiment


In [7]:
twitter_train['sentiment'] = pd.DataFrame([rename_sentiment(e) for e in twitter_train['sentiment']]).set_index(twitter_train.index)
twitter_validate['sentiment'] = pd.DataFrame([rename_sentiment(e) for e in twitter_validate['sentiment']]).set_index(twitter_validate.index)
twitter_test['sentiment'] = pd.DataFrame([rename_sentiment(e) for e in twitter_test['sentiment']]).set_index(twitter_test.index)

In [8]:
airline_train['sentiment'] = pd.DataFrame([rename_sentiment(e) for e in airline_train['sentiment']]).set_index(airline_train.index)
airline_validate['sentiment'] = pd.DataFrame([rename_sentiment(e) for e in airline_validate['sentiment']]).set_index(airline_validate.index)
airline_test['sentiment'] = pd.DataFrame([rename_sentiment(e) for e in airline_test['sentiment']]).set_index(airline_test.index)

In [9]:
apple_train['sentiment'] = pd.DataFrame([rename_sentiment(e) for e in apple_train['sentiment']]).set_index(apple_train.index)
apple_validate['sentiment'] = pd.DataFrame([rename_sentiment(e) for e in apple_validate['sentiment']]).set_index(apple_validate.index)
apple_test['sentiment'] = pd.DataFrame([rename_sentiment(e) for e in apple_test['sentiment']]).set_index(apple_test.index)

In [10]:
# cat the datasets
train_ds = pd.concat([twitter_train,airline_train,apple_train],axis=0)
validate_ds = pd.concat([twitter_validate,airline_validate,apple_validate],axis=0)
test_ds = pd.concat([twitter_test,airline_test,apple_test],axis=0)

In [11]:
bert_weights_name = 'bert-base-cased'
bert_tokenizer = BertTokenizer.from_pretrained(bert_weights_name)
bert_model = BertModel.from_pretrained(bert_weights_name)
# model = BertForSequenceClassification.from_pretrained(bert_weights_name)
# Unique values of sentiment
twitter_sentiment_labels = train_ds['sentiment'].unique()

In [12]:
train_ds.size, validate_ds.size, test_ds.size

(402588, 17112, 145548)

In [13]:
%%time
bert_experiment_full = sst.experiment(
    [train_ds], # 
    dataset.hf_cls_phi,
    dataset.fit_softmax_classifier,
    assess_dataframes=[validate_ds],
    vectorize=False)

              precision    recall  f1-score   support

    Negative      0.738     0.851     0.791      1294
     Neutral      0.673     0.602     0.635      1004
    Positive      0.672     0.563     0.613       554

    accuracy                          0.707      2852
   macro avg      0.695     0.672     0.680      2852
weighted avg      0.703     0.707     0.702      2852

CPU times: user 6h 56min 30s, sys: 2min 36s, total: 6h 59min 6s
Wall time: 1h 11min 30s


In [14]:
bert_experiment_full.keys()

dict_keys(['model', 'phi', 'train_dataset', 'assess_datasets', 'predictions', 'metric', 'scores'])

In [15]:
bert_experiment_full['scores']

[0.6796952992470061]

In [16]:
bert_experiment_full['metric']

'safe_macro_f1'

In [17]:
bert_experiment_full['model']

LogisticRegression(multi_class='ovr', solver='liblinear')

# Test BERT trained on Tweets on test set

In [18]:
def predict_one_bert(text):
    # List of tokenized examples:
    X = [bert_experiment_full['phi'](text)]
    # Standard `predict` step on a list of lists of str:
    preds = bert_experiment_full['model'].predict(X)
    # Be sure to return the only member of the predictions,
    # rather than the singleton list:
    return preds[0]

In [19]:
# %% time
# twitter_test['prediction'] = twitter_test['text'].apply(predict_one_bert)

In [20]:
# import importlib
# importlib.reload(sst)

In [21]:
%%time
bert_test = sst.evaluate(
    bert_experiment_full['model'],
    bert_experiment_full['phi'],
    assess_dataframes=[test_ds],
    vectorizer=bert_experiment_full['assess_datasets'][0]['vectorizer'],
    vectorize=False
)

              precision    recall  f1-score   support

    Negative      0.644     0.690     0.666      7789
     Neutral      0.625     0.650     0.637      9950
    Positive      0.630     0.536     0.579      6519

    accuracy                          0.633     24258
   macro avg      0.633     0.626     0.628     24258
weighted avg      0.632     0.633     0.631     24258

CPU times: user 2h 22min 46s, sys: 50.7 s, total: 2h 23min 37s
Wall time: 23min 56s


In [22]:
type(bert_test['predictions'][0])

numpy.ndarray

In [23]:
predictions_fname ='results/BERT_predictions_on_combined_neutral_twitter_test.csv'
df = bert_test['predictions'][0]
pd.DataFrame(df).to_csv(predictions_fname)

In [24]:
encoding_fname ='results/BERT_encodings_on_combined_neutral_twitter_test.csv'
encoded_test = bert_test['assess_datasets'][0]
pd.DataFrame(df).to_csv(encoding_fname)

In [25]:
predictions_df = pd.DataFrame(df)
predictions_df = predictions_df.set_index(test_ds.index)
predictions_df

,0
1,Neutral
12,Neutral
13,Neutral
17,Neutral
19,Positive
...,...
3847,Neutral
3852,Neutral
3869,Neutral
3871,Neutral


In [26]:
test_ds['BERT_sentiment'] = predictions_df

In [27]:
test_ds

,dataset,tweet_id,text,sentiment,entity,airline,BERT_sentiment
1,twitter_sentiment,1,I am coming to the borders and I will kill you...,Positive,2401.0,NaN,Neutral
12,twitter_sentiment,2,"Rock-Hard La Varlope, RARE & POWERFUL, HANDSOM...",Neutral,2403.0,NaN,Neutral
13,twitter_sentiment,3,"Rock-Hard La Varlope, RARE & POWERFUL, HANDSOM...",Neutral,2403.0,NaN,Neutral
17,twitter_sentiment,4,"I-Hard like me, RARE LONDON DE, HANDSOME 2011,...",Neutral,2403.0,NaN,Neutral
19,twitter_sentiment,5,this was the first Borderlands session in a lo...,Positive,2404.0,NaN,Positive
...,...,...,...,...,...,...,...
3847,twitter_apple,623499377,proof @apple does not use it's own products. f...,Negative,NaN,NaN,Neutral
3852,twitter_apple,623499382,RT @TeamCavuto: Protesters stage #DieIn protes...,Neutral,NaN,NaN,Neutral
3869,twitter_apple,623499399,Apple Is Warming Up To Social Media: Apple is ...,Neutral,NaN,NaN,Neutral
3871,twitter_apple,623499401,Apple Is Warming Up To Social Media: Apple is ...,Neutral,NaN,NaN,Neutral


In [28]:
test_predictions_fname ='results/BERT_predictions_added_to_combined_neutral_twitter_test.csv'
test_ds.to_csv(test_predictions_fname)

In [29]:
correct = test_ds[test_ds['sentiment'] == test_ds['BERT_sentiment']]

In [30]:
correct

,dataset,tweet_id,text,sentiment,entity,airline,BERT_sentiment
12,twitter_sentiment,2,"Rock-Hard La Varlope, RARE & POWERFUL, HANDSOM...",Neutral,2403.0,NaN,Neutral
13,twitter_sentiment,3,"Rock-Hard La Varlope, RARE & POWERFUL, HANDSOM...",Neutral,2403.0,NaN,Neutral
17,twitter_sentiment,4,"I-Hard like me, RARE LONDON DE, HANDSOME 2011,...",Neutral,2403.0,NaN,Neutral
19,twitter_sentiment,5,this was the first Borderlands session in a lo...,Positive,2404.0,NaN,Positive
29,twitter_sentiment,6,the biggest dissappoinment in my life came bac...,Negative,2405.0,NaN,Negative
...,...,...,...,...,...,...,...
3824,twitter_apple,623499354,Finally got my iPhone 6! @apple =) Such a reli...,Positive,NaN,NaN,Positive
3852,twitter_apple,623499382,RT @TeamCavuto: Protesters stage #DieIn protes...,Neutral,NaN,NaN,Neutral
3869,twitter_apple,623499399,Apple Is Warming Up To Social Media: Apple is ...,Neutral,NaN,NaN,Neutral
3871,twitter_apple,623499401,Apple Is Warming Up To Social Media: Apple is ...,Neutral,NaN,NaN,Neutral


In [31]:
incorrect = test_ds[test_ds['sentiment'] != test_ds['BERT_sentiment']]

In [32]:
incorrect

,dataset,tweet_id,text,sentiment,entity,airline,BERT_sentiment
1,twitter_sentiment,1,I am coming to the borders and I will kill you...,Positive,2401.0,NaN,Neutral
44,twitter_sentiment,9,Watch this epic striptease!.,Neutral,2408.0,NaN,Positive
86,twitter_sentiment,27,FUCK YESSSSSSSS.,Positive,2415.0,NaN,Negative
132,twitter_sentiment,39,Come meet one of the lovely Gaming Goddesses .,Positive,2423.0,NaN,Neutral
138,twitter_sentiment,40,How the hell are we into Halloween month alrea...,Neutral,2424.0,NaN,Negative
...,...,...,...,...,...,...,...
3731,twitter_apple,623499261,.@Apple and @Amazon refuse to make full federa...,Negative,NaN,NaN,Neutral
3770,twitter_apple,623499300,Our new @Technogym treadmills getting assemble...,Neutral,NaN,NaN,Negative
3830,twitter_apple,623499360,C'mon @Apple I'm ready to buy your watch. Hurr...,Negative,NaN,NaN,Positive
3835,twitter_apple,623499365,@afrobugeisha @Apple watch it!!!!!,Neutral,NaN,NaN,Positive


In [33]:
irrelevant = test_ds[test_ds['sentiment'] == 'Irrelevant']
irrelevant

,dataset,tweet_id,text,sentiment,entity,airline,BERT_sentiment


# Save Model

In [34]:
import pickle
model_fname = 'models/BERT_twitter_model_combined_neutral.sav'
pickle.dump(bert_experiment_full['model'], open(model_fname, 'wb'))

In [35]:
test_fname = 'results/BERT_test_combined_neutral.sav'
pickle.dump(bert_test, open(test_fname, 'wb'))